In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from os.path import exists, join
import sys
import copy

import scipy.io as sio
import importlib
import nnutils as utils
import rsatools as rsa

importlib.reload(utils)
importlib.reload(rsa)


<module 'rsatools' from '/home/jacobpri/git/RotspaceIT/notebooks/rsatools.py'>

In [8]:
arch = 'alexnet'
trained_on = 'object'
imageset = 'tripartite'
img_dim = 224
batch_size = 40
FDR_threshold = 0.05
overwrite = True

save_as = '.npy'
save_layer_rdvs = False
rdv_dist = 'correlation'
verbose = False
draw_plots = False

In [9]:
homedir = '/home/jacobpri/git/RotspaceIT/'
weight_dir = join(homedir,'data','d02_modeling','weights')

network = arch + '-' + trained_on
FDR_str = str(FDR_threshold).replace('.','-')
activation_savedir = join(homedir,'data','d02_modeling','activations',network,'localizer',imageset,'dim'+str(img_dim))
os.makedirs(activation_savedir,exist_ok=True)

In [10]:
imageset_dir = join(homedir,'imagesets','localizer',imageset)
assert(exists(imageset_dir))

floc_savedir = join(homedir,'data','d02_modeling','selectivity',network, 'localizer',imageset,'dim'+str(img_dim))
os.makedirs(floc_savedir,exist_ok=True)

utils.reproducible_results(365)


In [11]:
floc_str = f'layer_pref_dicts_FDR_p_{FDR_str}.npy'
floc_fullsave_fn = join(floc_savedir,floc_str)
print(floc_fullsave_fn)

if exists(floc_fullsave_fn):
    print('preference dict already exists. loading...')
    pref_dicts = np.load(floc_fullsave_fn,allow_pickle=True).item()
else:
    print('preference dict not found')

/home/jacobpri/git/RotspaceIT/data/d02_modeling/selectivity/alexnet-object/localizer/tripartite/dim224/layer_pref_dicts_FDR_p_0-05.npy
preference dict already exists. loading...


In [13]:
exp_imageset = 'size-manip'
min_n_units = 5
exp_activation_savedir = join(homedir,'data','d02_modeling','activations',network,'experimental',exp_imageset,'dim'+str(img_dim))
assert(exists(exp_activation_savedir))

output_act_dir = join(homedir,'outputs','cat-size-manip','activations')
output_fig_dir = join(homedir,'outputs','cat-size-manip','figures')
output_rdm_dir = join(homedir,'outputs','cat-size-manip','rdms')

for d in [output_act_dir, output_fig_dir, output_rdm_dir]:
    if not exists(d):
        os.makedirs(d,exist_ok=True)

for layer in list(pref_dicts.keys()):
    
    layer_ = layer.split('_')[1]
    
    Y = utils.load_batched_activations(exp_activation_savedir, layer, batch_size, reshape_to_2D = True)
    
    pref_large_obj_idx = pref_dicts[layer]['domain_idx'][1]
    pref_small_obj_idx = pref_dicts[layer]['domain_idx'][2]
    
    Y_large = Y[:,pref_large_obj_idx]
    Y_small = Y[:,pref_small_obj_idx]
    
    fn_large = join(output_act_dir, f'pref_BigInanimate_{layer}_activations.npy')
    fn_small = join(output_act_dir, f'pref_SmallInanimate_{layer}_activations.npy')
    
    np.save(fn_large, Y_large)
    np.save(fn_small, Y_small)
    
    if len(pref_large_obj_idx) > min_n_units and len(pref_small_obj_idx) > min_n_units:
        
        Y_large_rdm = rsa.rdm(Y[:,pref_large_obj_idx])
        Y_small_rdm = rsa.rdm(Y[:,pref_small_obj_idx])
        
        plt.figure(figsize=(18,14))
        plt.subplot(221)
        plt.imshow(Y_large,aspect='auto')
        plt.title(f"activations - {layer_}:\n pref BigInanimate (n={len(pref_large_obj_idx)} units)")
        plt.colorbar()
        plt.xlabel('units')
        plt.ylabel('imgs')
        plt.subplot(222)
        plt.imshow(Y_large_rdm)
        plt.title(f"rdm - {layer_}:\n pref BigInanimate (n={len(pref_large_obj_idx)} units)")
        plt.colorbar()
        plt.xlabel('imgs')
        plt.ylabel('imgs')
        plt.subplot(223)
        plt.imshow(Y_small,aspect='auto')
        plt.title(f"activations - {layer_}:\n pref SmallInanimate (n={len(pref_small_obj_idx)} units)")
        plt.colorbar()
        plt.xlabel('units')
        plt.ylabel('imgs')
        plt.subplot(224)
        plt.imshow(Y_small_rdm)
        plt.title(f"rdm - {layer_}:\n pref SmallInanimate (n={len(pref_small_obj_idx)} units)")
        plt.colorbar()
        plt.xlabel('imgs')
        plt.ylabel('imgs')
        plt.savefig(join(output_fig_dir,f'{layer}_bigsmall_pref_unit_acts_and_rdms.png'))
        plt.close()
        
        fn_large = join(output_rdm_dir, f'pref_BigInanimate_{layer}_rdm.npy')
        fn_small = join(output_rdm_dir, f'pref_SmallInanimate_{layer}_rdm.npy')
    
        np.save(fn_large, Y_large_rdm)
        np.save(fn_small, Y_small_rdm)
                
        Y_large_means = np.mean(Y_large,axis=1)
        Y_small_means = np.mean(Y_small,axis=1)
        
        plt.figure(figsize=(12,8))
        plt.subplot(121)
        sns.boxplot(data=[Y_large_means[:108],
                     Y_large_means[108:216],
                     Y_large_means[216:324],
                     Y_large_means[324:]])

        plt.xticks(np.arange(4),np.array(['BigManip','BigNonManip','SmallManip','SmallNonManip']),rotation=45)
        plt.title(f"{layer_}:\n pref BigInanimate (n={len(pref_large_obj_idx)} units)")
        plt.ylabel('mean activation')
        plt.subplot(122)
        sns.boxplot(data=[Y_small_means[:108],
                     Y_small_means[108:216],
                     Y_small_means[216:324],
                     Y_small_means[324:]])
        plt.xticks(np.arange(4),np.array(['BigManip','BigNonManip','SmallManip','SmallNonManip']),rotation=45)
        plt.title(f"{layer_}:\n pref SmallInanimate (n={len(pref_small_obj_idx)} units)")
        plt.ylabel('mean activation')
        
        fig_fn = join(output_fig_dir, f'{layer}_size-manip_categ_univariate.png')

        plt.savefig(fig_fn)
        plt.close()
        